In [1]:
import gymnasium   #
import numpy as np
import CoppeliaSim_Gym   # Entorno personalizado
import stable_baselines3   # Biblioteca RL
import torch as th   # para modelo CNN
import torch.nn as nn
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env # Verificar el entorno
from stable_baselines3.common.vec_env import VecFrameStack, VecNormalize # Envoltorio para imagen
from stable_baselines3.sac.policies import MlpPolicy, CnnPolicy # Arquitectura de RN
from stable_baselines3 import A2C, TD3, DQN, DDPG, PPO, SAC, HER # algoritmo de entrenamiento
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.utils import get_system_info
#get_system_info(True)

In [2]:
# set the device
use_cuda = False
if use_cuda:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
else:
    device = th.device("cpu")
print(device)

cpu


In [3]:
# Inicializar el entorno
ENV_NAME = "CoppeliaSim_Gym/GymCoppManR-v0"
env = gymnasium.make(ENV_NAME) 
# check_env(env, warn=True) # si funciona
# env = VecFrameStack(env, n_stack=4)

Conectado al servidor API remoto


In [4]:
# Arquitectura CNN personalizada
class CustomCNN(BaseFeaturesExtractor):
    """
    :param observation_space: (gym.Space)
    :param features_dim: (int) Number of features extracted.
        This corresponds to the number of unit for the last layer.
    """
    def __init__(self, observation_space: spaces.Box, joint_dim: int = 7):
        super().__init__(observation_space, joint_dim)
        
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Flatten(),
            nn.ReLU(),
        )
        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, joint_dim), nn.ReLU())
        
    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))
    
policy_kwargs = dict(
    features_extractor_class = CustomCNN,
    features_extractor_kwargs = dict(joint_dim=7),
)
fc = policy_kwargs
print(fc)

{'features_extractor_class': <class '__main__.CustomCNN'>, 'features_extractor_kwargs': {'joint_dim': 7}}


In [5]:
# Arquitectura CNN prueba
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Box, joint_dim: int = 7):
        super().__init__(observation_space, joint_dim)
        
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 16, kernel_size=8, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=2, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.ReLU(),
            nn.Linear(256, joint_dim),
        )
        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]
        self.linear = nn.Sequential(nn.Linear(n_flatten, joint_dim), nn.ReLU())
    
    # Pasar el estado en el tiempo t a través de la red para obtener Q(s,a)    
    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))
    
policy_kwargs = dict(
    features_extractor_class = CustomCNN,
    features_extractor_kwargs = dict(joint_dim=7),
)
fc = policy_kwargs
print(fc)

{'features_extractor_class': <class '__main__.CustomCNN'>, 'features_extractor_kwargs': {'joint_dim': 7}}


In [3]:
# Arquitectura simple cnn
class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Box, joint_dim: int = 7):
        super().__init__(observation_space, joint_dim)
        
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 16, kernel_size=8, stride=3, padding=3),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=8, stride=3, padding=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.ReLU(),
        )
        self.linear = nn.Sequential(nn.Linear(n_flatten, joint_dim), nn.ReLU())
    
    # Pasar el estado en el tiempo t a través de la red para obtener Q(s,a)    
    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))
    
policy_kwargs = dict(
    features_extractor_class = CustomCNN,
    features_extractor_kwargs = dict(joint_dim=7),
)
fc = policy_kwargs
print(fc)

{'features_extractor_class': <class '__main__.CustomCNN'>, 'features_extractor_kwargs': {'joint_dim': 7}}


In [5]:
# Definir y entrenar el agente
model = A2C('CnnPolicy', env, policy_kwargs=policy_kwargs, verbose=1, device="auto", tensorboard_log='./A2C_tensorboard/')
model.learn(total_timesteps = 5000)
model.save("modeloA2C") # Guardar modelo DRL

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to ./A2C_tensorboard/A2C_29
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.844589352607727 -0.0011013359762728214 0.06420253217220306
Distancia previa : 0.21 m
Distancia actual: 0.21 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  -11.0
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.28456205129623413 0.09206673502922058 0.24310721457004547
Distancia previa : 0.80 m
Distancia actual: 0.80 m
Recompensa distancia: 7.439973984102455e-05
Recompensa longitud brazo: 13.2203720703125
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  2.220446470052341
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.19750870764255524 0.16177238523960114 0.1650625765323639
Distancia previa : 0.88 m
Distancia actual: 0.88 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  48.07515063275241
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.28511863946914673 0.09192659705877304 0.24175314605236053
Distancia previa : 0.80 m
Distancia actual: 0.80 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  37.07515063275241
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.1944587677717209 0.1601279377937317 0.15835756063461304
Distancia previa : 0.88 m
Distancia actual: 0.88 m
Recompensa distancia: 0.0024938881368037347
Recompensa longitud brazo: 17.64879687

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.32891860604286194 0.21654313802719116 0.21411478519439697
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  89.1721084342937
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.3288072645664215 0.2164439857006073 0.21399597823619843
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: 0.0001480409514086345
Recompensa longitud brazo: 13.452693359375
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  102.62494983462011
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.3570407032966614 0.1951116919517517 0.18140698969364166
Distancia previa : 0.73 m
Distancia actual: 0.76 m
Recompensa distancia: 0.021468794373282973
Recompensa lo

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.32892829179763794 0.2167033702135086 0.21411874890327454
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  24.05969911043706
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.328777551651001 0.21647073328495026 0.2143537402153015
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  13.059699110437059
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.3287561237812042 0.21647347509860992 0.21394480764865875
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: 0.00014853881285437076
Recompensa longitud brazo: 8.634426757

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.32877615094184875 0.21648472547531128 0.21399971842765808
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: 0.0002488533265659898
Recompensa longitud brazo: 16.1659287109375
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  128.28893278784923
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.32786881923675537 0.2166048288345337 0.2160380482673645
Distancia previa : 0.78 m
Distancia actual: 0.78 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  117.28893278784923
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.2656709849834442 0.2016105204820633 0.16754081845283508
Distancia previa : 0.82 m
Distancia actual: 0.82 m
Recompensa distancia: -1.0
Recompensa longitud brazo:

...

Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: 3.296451518974841e-06
Recompensa longitud brazo: 14.00813671875
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6367.02829500296
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079564929008484 -0.0009560096077620983 0.3687427341938019
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: 1.4825169900034574e-06
Recompensa longitud brazo: 14.236134765625
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6381.264431251102
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079567909240723 -0.0009552875417284667 0.3687421679496765
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6370.264431

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079691886901855 -0.0009041039156727493 0.36874040961265564
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: 2.4959806624869074e-06
Recompensa longitud brazo: 11.212166015625
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6512.954583137792
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079578042030334 -0.0009543176856823266 0.36873266100883484
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6501.954583137792
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079601287841797 -0.0009431213838979602 0.368736207485199
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: -1.0
Recompensa longitud 

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079620361328125 -0.0009306009160354733 0.3687436878681183
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: 4.0105320446848225e-06
Recompensa longitud brazo: 8.970693359375
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6540.930555535527
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.707956850528717 -0.0009603676735423505 0.36873048543930054
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: -1.0
Recompensa longitud brazo: -10.0
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6529.930555535527
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079567909240723 -0.0009602864738553762 0.3687308430671692
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: -1.0
Recompensa longitud br

pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079601883888245 -0.0009431366925127804 0.36873602867126465
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: 2.7596041529132087e-06
Recompensa longitud brazo: 11.8431123046875
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6484.814299094119
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079620957374573 -0.0009334954665973783 0.3687434494495392
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia: 7.675436629273058e-06
Recompensa longitud brazo: 17.058130859375
El efector y el objeto no están colisionando
Recompensa Colision: 0.0
Recompensa Total:  6501.87243762893
Contador:  1
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.7079567313194275 -0.0009603392682038248 0.3687308430671692
Distancia previa : 0.48 m
Distancia actual: 0.48 m
Recompensa distancia

KeyboardInterrupt: 

In [6]:
del model # si se entrena agregar, cc solo cargar modelo
model = A2C.load("modeloA2C", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [10]:
model.policy # mostrar arq del modelo

ActorCriticCnnPolicy(
  (features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU()
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU()
      (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (9): Flatten(start_dim=1, end_dim=-1)
      (10): ReLU()
    )
    (linear): Sequential(
      (0): Linear(in_features=32768, out_features=7, bias=True)
      (1): ReLU()
    )
  )
  (pi_features_extractor): CustomCNN(
    (cnn): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, 

In [7]:
# Evaluar el modelo entrenado
ep_reward = 0
obs, inf = env.reset()
print('obs', obs.shape)
for episode in range(25):
    env.render()
    action, _state = model.predict(obs, deterministic=False) # ,deterministic=True
    n_obs, reward, done, tru, info = env.step(action)
    #print(f'Obs: {obs} reward: {reward} done: {done} info: {info}')
    #print(f'Acciones: {action} Estado: {reward}')
    ep_reward += reward
    print(f'reward: {reward} done: {done} act: {action}')
    obs = n_obs
env.close()

obs (85, 85, 1)
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.1869720220565796 0.11968936771154404 0.09277841448783875
Distancia previa : 0.88 m
Distancia actual: 0.84 m
Recompensa distancia: -1.00 m
Recompensa longitud brazo: -10.00 m
El efector y el objeto no están colisionando
Recompensa colision  -1.0
Recompensa Total:  -904.9158974763964
Episodio:  1
reward: -904.9158974763964 done: False act: [ 0.09022918 -0.9297829  -1.         -1.          1.         -0.69811785
 -1.        ]
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.6406646370887756 0.07588031888008118 0.08093105256557465
Distancia previa : 0.42 m
Distancia actual: 0.68 m
Recompensa distancia: 0.26 m
Recompensa longitud brazo: 16.59 m
El efector y el objeto no están colisionando
Recompensa colision  -1.0
Recompensa Total:  -889.070971006334
Episodio:  2
reward: -889.070971006334 done: False act: [ 0.67481184 -1.          0.7966881  -1.          0.95136565  0.2

El efector y el objeto no están colisionando
Recompensa colision  -1.0
Recompensa Total:  -838.0926320177742
Episodio:  17
reward: -838.0926320177742 done: False act: [-0.53489554 -1.          0.08292097 -1.          0.59791493  0.65944874
  1.        ]
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.186964213848114 -0.1299651712179184 0.18825341761112213
Distancia previa : 0.89 m
Distancia actual: 0.78 m
Recompensa distancia: -1.00 m
Recompensa longitud brazo: -10.00 m
El efector y el objeto no están colisionando
Recompensa colision  -1.0
Recompensa Total:  -850.0926320177742
Episodio:  18
reward: -850.0926320177742 done: False act: [-0.36266598 -0.61075866 -0.5970197  -0.8573866  -0.76148957 -0.7228162
  0.785528  ]
pos esfera:  1.0499999523162842 0.0 0.02500000037252903
pos efector final:  0.6053311228752136 0.12224465608596802 0.20040379464626312
Distancia previa : 0.49 m
Distancia actual: 1.11 m
Recompensa distancia: 0.61 m
Recompensa longitud brazo: 